# Azure AI Agent service - Connected agents

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
import os, time
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import FunctionTool, RequiredFunctionToolCall, ToolOutput, SubmitToolOutputsAction
import json
import datetime
from typing import Any, Callable, Set, Dict, List, Optional
from dotenv import load_dotenv

## Settings

In [2]:
load_dotenv(".env")

False

In [3]:
endpoint = os.getenv("PROJECT_ENDPOINT")
credential = DefaultAzureCredential()

project_client = AIProjectClient(endpoint=endpoint, credential=credential)

## Function definition

### Fetch Weather Agent

In [4]:
def fetch_weather(location: str) -> str:
    """
    Fetches the weather information for the specified location.

    :param location: The location to fetch weather for.
    :return: Weather information as a JSON string.
    """
    # Mock weather data for demonstration purposes
    mock_weather_data = {"New York": "Sunny, 25°C", "London": "Cloudy, 18°C", "Tokyo": "Rainy, 22°C"}
    weather = mock_weather_data.get(location, "Weather data not available for this location.")
    return json.dumps({"weather": weather})

# Define user functions
user_functions = {fetch_weather}

In [ ]:
# Retrieve the project endpoint from environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]
model_name = os.environ["MODEL_DEPLOYMENT_NAME"]
# Initialize the AIProjectClient
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential()
)

# Initialize the FunctionTool with user-defined functions
functions = FunctionTool(functions=user_functions)

with project_client:
    # Create an agent with custom functions
    agent = project_client.agents.create_agent(
        model=model_name,
        name="my-agent",
        instructions="You are a helpful agent",
        tools=functions.definitions,
    )
    print(f"Created agent, ID: {agent.id}")

    # Create a thread for communication
    thread = project_client.agents.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Send a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",
        content="How is weather in New York?",
    )
    print(f"Created message, ID: {message['id']}")

    # Create and process a run for the agent to handle the message
    run = project_client.agents.runs.create(thread_id=thread.id, agent_id=agent.id)
    print(f"Created run, ID: {run.id}")

    # Poll the run status until it is completed or requires action
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = project_client.agents.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                project_client.agents.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_outputs = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredFunctionToolCall):
                    try:
                        print(f"Executing tool call: {tool_call}")
                        output = functions.execute(tool_call)
                        tool_outputs.append(
                            ToolOutput(
                                tool_call_id=tool_call.id,
                                output=output,
                            )
                        )
                    except Exception as e:
                        print(f"Error executing tool_call {tool_call.id}: {e}")
            if tool_outputs:
                project_client.agents.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)

    print(f"Run completed with status: {run.status}")

    # Fetch and log all messages from the thread
    messages = project_client.agents.messages.list(thread_id=thread.id)
    for message in messages:
        print(f"Role: {message['role']}, Content: {message['content']}")

   # Clean up resources by deleting the agent 
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

Created agent, ID: asst_JsYHEzAhn6l0vK1lNdrRcljH
Created thread, ID: thread_MUSdTTWur69vcOXE28tcRIRz
Created message, ID: msg_PfbEd1e4FidBjsWUqtvyXRiG
Created run, ID: run_9hIJJCBCr2x02XgQdjdwxYlA
Executing tool call: {'id': 'call_y2eQVHVcKdOyYLJXtSIeEDPa', 'type': 'function', 'function': {'name': 'fetch_weather', 'arguments': '{"location":"New York"}'}}
Run completed with status: RunStatus.COMPLETED
Role: assistant, Content: [{'type': 'text', 'text': {'value': 'The weather in New York is sunny with a temperature of 25°C.', 'annotations': []}}]
Role: user, Content: [{'type': 'text', 'text': {'value': 'How is weather in New York?', 'annotations': []}}]


ValueError: HTTP transport has already been closed. You may check if you're calling a function outside of the `with` of your client creation, or if you called `close()` on your client already.